In [1]:
from DataManagement.data_manager import DataManager
from DataManagement.coin_data import CoinData
from TransferEntropy.transfer_entropy import TransferEntropy
from constants import ATR_ABNORMALITY_COLUMN
# libs
import pandas as pd
import numpy as np

# plotting
import seaborn as sn
import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (70,30)

# mutual information
from sklearn.feature_selection import mutual_info_classif

In [2]:
# coin_symbol_list = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "LTCUSDT", "BNBUSDT", "XRPUSDT"]
coin_symbol_list = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "LTCUSDT"]
feature = "close_pct"
kline_size = "1h"
main_coin_symbol = "BTCUSDT"
transfer_entropy_lookback = 1


In [3]:
data_manager = DataManager()
transfer_entropy = TransferEntropy(lookback=transfer_entropy_lookback)

In [4]:
main_1h_df = data_manager.get_historical_data_CoinData(main_coin_symbol, kline_size="1h").df
main_abnromal_timestamps = main_1h_df[main_1h_df[ATR_ABNORMALITY_COLUMN] == 1].index

In [5]:
coin_data_list = [data_manager.get_historical_data_CoinData(coin_symbol, kline_size="1m") for coin_symbol in coin_symbol_list]
one_hour_coin_data_list = [data_manager.get_historical_data_CoinData(coin_symbol, kline_size="1h") for coin_symbol in coin_symbol_list]

# Join close_pct columns of all coins togther

In [6]:
def join_dataframes(coin_data_list, shift):
    df = pd.DataFrame(index=coin_data_list[0].df.index)

    for coin_data in coin_data_list:
        df = df.join(coin_data.df.shift(shift)[feature], how="outer", rsuffix=f"_{coin_data.coin_symbol}", on="timestamp")

    df.rename(columns={"close_pct":f"close_pct_{coin_data_list[0].coin_symbol}"}, inplace=True)    
    df.dropna(inplace=True)
    df.drop(columns=["timestamp"])
    df.set_index("timestamp", inplace=True)
    return df

In [7]:
one_min_df = join_dataframes(coin_data_list=coin_data_list, shift=0)
one_hour_df = join_dataframes(one_hour_coin_data_list, shift=0)
one_hour_df = data_manager.add_noise_to_df(one_hour_df)

In [8]:
one_min_df

,close_pct_BTCUSDT,close_pct_ETHUSDT,close_pct_ADAUSDT,close_pct_LTCUSDT
timestamp,,,,
2018-04-17 04:03:00,0.000410,0.000000,-0.033037,0.000998
2018-04-17 04:04:00,0.000210,0.000374,0.021373,-0.001840
2018-04-17 04:05:00,-0.000368,0.000000,0.000000,-0.000077
2018-04-17 04:06:00,0.000415,-0.000020,0.012338,0.000230
2018-04-17 04:07:00,-0.000383,0.000788,-0.011298,0.000614
...,...,...,...,...
2021-06-30 10:39:00,0.001242,0.000965,0.001122,0.001208
2021-06-30 10:40:00,-0.002035,-0.002306,-0.001419,-0.003194
2021-06-30 10:41:00,-0.000905,-0.000366,-0.000599,-0.000641


In [23]:
def get_transfer_entropy_list(df):
    lookback = 4
    num_cols = df.shape[1]
    temp_result = np.ones(shape=(num_cols, num_cols))
    candle_series = [list((df[col]).resample("1d")) for col in df.columns]
    a = np.array(candle_series)
    # print(a.shape)
    temp_result = np.empty(shape=a.shape[1])
    for i in range(a.shape[1]):
        if i <= lookback:
            continue
        temp_candle_series = a[:, i-lookback:i, :]
        for i in range(num_cols):
            for j in range(num_cols):
                # print(temp_candle_series[i, :, 1][0])
                temp_result[i,j] = transfer_entropy.get_transfer_entropy(temp_candle_series[i,: ,1], temp_candle_series[j, :, 1])
        
        result[i] = temp_result
        print(temp_result)
        print(result.shape)
        return
    # return result

In [24]:
get_te_new(one_min_df)

ValueError: Can only compare identically-labeled Series objects

In [40]:
resampled_list = np.array(one_min_df.resample("1h"))

In [45]:
get_transfer_entropy_matrix(one_min_df, resampled_list)

[                     close_pct_BTCUSDT  close_pct_ETHUSDT  close_pct_ADAUSDT  \
timestamp                                                                      
2018-04-17 04:03:00           0.000410           0.000000          -0.033037   
2018-04-17 04:04:00           0.000210           0.000374           0.021373   
2018-04-17 04:05:00          -0.000368           0.000000           0.000000   
2018-04-17 04:06:00           0.000415          -0.000020           0.012338   
2018-04-17 04:07:00          -0.000383           0.000788          -0.011298   
2018-04-17 04:08:00           0.000463          -0.000236          -0.004908   
2018-04-17 04:09:00          -0.000131           0.000000           0.005158   
2018-04-17 04:10:00          -0.000357          -0.000177          -0.007304   
2018-04-17 04:11:00          -0.000016          -0.000098           0.007056   
2018-04-17 04:12:00          -0.000636          -0.000079           0.000300   
2018-04-17 04:13:00          -0.001770 

In [17]:
# def get_mean_transfer_entropy(df):
#     # df is one_min_df
#     transfer_entropy.get_transfer_entropy(df.iloc[:,i].to_numpy(), df.iloc[:,j].to_numpy())
    

#     return 3

def get_transfer_entropy_matrix(df):
    num_cols = df.shape[1]
    result = np.ones(shape=(num_cols, num_cols))
    skipping_counter = 0
    resampled_list = np.array(df.resample("1h"))
    temp_result = np.ones(shape=(num_cols, num_cols))
    for timestamp, temp_df in resampled_list:
        
        if temp_df.empty:
            skipping_counter += 1
            continue
        
        transfer_entropy_lookback
        
    
        # for i in range(num_cols):
        #     for j in range (num_cols):
        #         if i==j:
        #             continue
                
        #         result[i, j] = get_mean_transfer_entropy(df)

    # return pd.DataFrame(data=result, columns=df.columns, index=df.columns)

# Transfer Entropy

In [ ]:
def get_transfer_entropy_matrix(df : pd.DataFrame, method: str = None, is_symetric = False):
    num_cols = df.shape[1]
    result = np.ones(shape=(num_cols, num_cols))

    if method == "transfer_entropy" or method is None:
        method = transfer_entropy.get_transfer_entropy

    if is_symetric:
        if method is None:
            method = "spearman"
        return df.corr(method=method)


    for i in range(num_cols):
        for j in range(num_cols):
            if i==j:
                continue
            result[i,j] = method(df.iloc[:, i].to_numpy(), df.iloc[:, j].to_numpy())
    
    return pd.DataFrame(data=result, columns=df.columns, index=df.columns)

In [ ]:
def get_transfer_entropy_correl_matrix_list(df):
    window_df_list = list(df.rolling(window=transfer_entropy_lookback))

    alltime_correlation_matrix_list, abnormal_correlation_matrix_list = [],[]
    i = 0
    skipping_counter = 0
    for temp_df in window_df_list: # every df is the 1min data from that hour.
        timestamp = temp_df.index[0]
        if temp_df.empty:
            skipping_counter += 1
            continue
        try:
            temp_correl_matrix = get_transfer_entropy_matrix(df=temp_df).to_numpy()
            
        except:
            skipping_counter += 1
            continue
        
        if np.any(np.isnan(temp_correl_matrix)):
            skipping_counter += 1
            continue
        
        if timestamp in main_abnromal_timestamps:
            abnormal_correlation_matrix_list.append(temp_correl_matrix)
        
        alltime_correlation_matrix_list.append(temp_correl_matrix)

    # transfrom to np.array
    abnormal_correlation_matrix_list = np.array(abnormal_correlation_matrix_list)
    alltime_correlation_matrix_list = np.array(alltime_correlation_matrix_list)
    
    # print skipping precent
    print(f"skipping precent {(skipping_counter/len(window_df_list))*100} %")
    return alltime_correlation_matrix_list, abnormal_correlation_matrix_list

In [ ]:
def get_correl_matrix_list(df, method = 'spearman'):
    if method == 'transfer_entropy':
        print("transfer_entropy")
        return get_transfer_entropy_correl_matrix_list(df)

    resampled_list = list(df.resample(rule="1h"))
    alltime_correlation_matrix_list, abnormal_correlation_matrix_list = [],[]

    skipping_counter = 0
    for timestamp, temp_df in resampled_list: # every df is the 1min data from that hour.
        
        if temp_df.empty:
            skipping_counter += 1
            continue

        try:

            temp_correl_matrix = temp_df.corr(method=method).to_numpy() # df of correlation between columns. 4x4
            
        except:
            skipping_counter += 1
            continue
        
        if np.any(np.isnan(temp_correl_matrix)):
            skipping_counter += 1
            continue
        
        if timestamp in main_abnromal_timestamps:
            abnormal_correlation_matrix_list.append(temp_correl_matrix)
        
        alltime_correlation_matrix_list.append(temp_correl_matrix)

    
    # transfrom to np.array
    abnormal_correlation_matrix_list = np.array(abnormal_correlation_matrix_list)
    alltime_correlation_matrix_list = np.array(alltime_correlation_matrix_list)
    
    # print skipping precent
    print(f"skipping precent {(skipping_counter/len(resampled_list))*100} %")
    return alltime_correlation_matrix_list, abnormal_correlation_matrix_list

# correl

In [ ]:
suptitle_fontsize = 40
font_scale = 3

In [ ]:
def plot_heatmap(correl_df, title, shift = 0):
    mean_corr_df = pd.DataFrame(index=coin_symbol_list, columns=coin_symbol_list, data=correl_df)

    fig_size = max(len(df.columns), 10)
    sn.set(rc={"figure.figsize":tuple([fig_size]*2)})
    title = f"Settings - feature:{feature} | time-frame:{kline_size} | main-coin:{main_coin_symbol} | shift:{shift}\n" + title

    sn.heatmap(mean_corr_df, annot=True).set(title=title);

def plot_heatmap_on_axes(data, title, axes):
    fig_size = 10
    sn.set(rc={"figure.figsize":tuple([fig_size]*2)}, font_scale = font_scale)
    
    size = data.shape[0]
    mean_corr_df = pd.DataFrame(index=coin_symbol_list[:size], columns=coin_symbol_list[:size], data=data)
    sn.heatmap(mean_corr_df, annot=True, ax=axes).set(title=title);

In [ ]:
def plot_heatmap_master(df, correl_method = 'spearman', shift = 0):
    alltime_correl_matrix_list, abnormal_correl_matrix_list = get_correl_matrix_list(df, method=correl_method)

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(70,20))
    fig_title = f"Settings - feature:{feature} | time-frame:{kline_size} | main-coin:{main_coin_symbol} | shift:{shift}"
    fig.suptitle(fig_title, fontsize = suptitle_fontsize)
    
    alltime_correl_mean = alltime_correl_matrix_list.mean(axis=0)
    abnormal_correl_mean = abnormal_correl_matrix_list.mean(axis=0)
    
    title_data_dict = {
        f"alltime Mean of 1h {correl_method}": alltime_correl_mean, 
        f"abnormal Mean of 1h {correl_method}":abnormal_correl_mean, 
        "abnromal minus alltime": abnormal_correl_mean-alltime_correl_mean}
    for i, (title, data) in enumerate(title_data_dict.items()):
        plot_heatmap_on_axes(data=data, title=title, axes=axes[i])
    fig.tight_layout()
    filename = "correlation_research_plot.jpg"
    fig.savefig(filename)

In [ ]:
# plot_heatmap_master(one_hour_df, correl_method='transfer_entropy')

In [ ]:
plot_heatmap_master(test_df, correl_method='transfer_entropy')